# core

> Fill in a module description here

In [3]:
#| default_exp core

In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
import json
import requests
import os
import docker
import typing
import tarfile
import jupytext
import sched
import time
import importlib.util
import inspect
import ast
import subprocess

In [6]:
#| hide
from fastcore.test import *

In [7]:
#| export
class Task(typing.TypedDict):
    job_run_id: int
    name: str
    image: str | None
    code_nb_path: str
    applets_nb_path: str
    input: dict

def _get_source(task: Task, root: str):
    spec = importlib.util.spec_from_file_location(
        "workflow", f"{root}/{task['code_nb_path']}")
    workflow_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(workflow_module)
    # getattr returns a DunderFunc
    func_source = inspect.getsource(getattr(workflow_module, task['name']).func)
    func_body = ast.parse(func_source).body[0].body
    if not isinstance(func_body, list):
        return ast.unparse(func_body)
    func_body_drop_return = [el.value if isinstance(el, ast.Return) else el for el in func_body]
    unparsed = map(ast.unparse, func_body_drop_return)
    return "\n".join(unparsed)

In [8]:
#| hide
test_eq(_get_source({'code_nb_path': 'yo.py', 'name': 'task_one'}, "./test_files"), "1")
test_eq(_get_source({'code_nb_path': 'yo.py', 'name': 'task_two'}, "./test_files"), "1")
test_eq(_get_source({'code_nb_path': 'yo.py', 'name': 'task_three'}, "./test_files"), "x = 1\nx")
test_eq(_get_source({'code_nb_path': 'yo.py', 'name': 'task_four'}, "./test_files"), "x = 1\ny = 2")

In [9]:
#| export
def get_imports_source(path):
    with open(path) as f:
      source = f.read()
      parsed_source = ast.parse(source)
      return ast.unparse([n for n in ast.walk(parsed_source) if isinstance(n, (ast.Import, ast.ImportFrom))])

In [10]:
#| hide
test_eq(get_imports_source("test_files/job_8.py"), "from data_chimp_executor import execute as dchimp\nimport pandas as pd")

In [26]:
#| export
# TODO: The func list arg generation from input
# is going to have to be more robust
def _input_value_to_arg_value(value):
    return f"'value'" if isinstance(value, str) else value

# TODO: All this templating is horrible
def _task_to_script(task, automations, root):
    task_source = _get_source(task, root)
    imports_source = get_imports_source(f"{root}/{task['code_nb_path']}")
    assign_targets = ", ".join(task['outputs'])
    task_args = '**inputs' if len(task['inputs']) > 0 else '' 
    return f"""\
# %%
{imports_source}
import {task['wf_name']}
import pickle
inputs = {{}}
input_names = json.loads('{json.dumps(task['inputs'])}')
for input in input_names:
    with open(f'task_storage/{{input}}') as f:
        inputs[input] = pickle.load(f)

{assign_targets} = {task['wf_name']}.{task['name']}({task_args})
output_names = json.loads('{json.dumps(task['outputs'])}')
for output in output_names:
    with open(f'task_storage/{{output}}') as f:
        pickle.dump(output, f)
{assign_targets}

# %%
import json
dchimp.on_execute_cell('''{json.dumps({"code": task_source})}''', '{json.dumps(automations)}', globals() | inputs)
"""

In [27]:
actual = _task_to_script({
    'wf_name': 'my_workflow', 
    'name': 'task_one', 
    'code_nb_path': 'yo.py', 
    'inputs': ['arg_1', 'arg_2'],
    'outputs': ['Y']
  },
  [{
    "id": "plots",
    "code": "df.plot.hist()"
  }],
  'test_files'
)
  
test_eq(actual, """\
# %%
from data_chimp_executor import execute as dchimp
import my_workflow
import pickle
inputs = {}
input_names = json.loads('["arg_1", "arg_2"]')
for input in input_names:
    with open(f'task_storage/{input}') as f:
        inputs[input] = pickle.load(f)

Y = my_workflow.task_one(**inputs)
output_names = json.loads('["Y"]')
for output in output_names:
    with open(f'task_storage/{output}') as f:
        pickle.dump(output, f)
Y

# %%
import json
dchimp.on_execute_cell('''{"code": "1"}''', '[{"id": "plots", "code": "df.plot.hist()"}]', globals() | inputs)
""")

In [92]:
#| export
def _update_task_status(host, task, status):
    requests.post(
        f"{host}/updateTask/{task['job_run_id']}",
        json={
            'task_name': task['name'],
            'status': status
        },
        headers={'x-token': os.environ.get('CHIMP_TOKEN')}
    ).raise_for_status()

In [27]:
#| export
def _get_automations(path: str, root: str) -> list:
    node = jupytext.read(f"{root}/{path}")
    return [
        {"id": "", "code": cell['source']}
        for cell in node.cells
        if cell['cell_type'] == "code" and "dchimp.ignore" not in cell['metadata'].get('tags', [])
    ]

In [29]:
test_eq(_get_automations("yo.ipynb", "./test_files"), [{'id': '', 'code': '#| eval: false\ndf.describe()'}])

In [ ]:
#| export
root = None
def _execute():
    if os.environ.get('CHIMP_TOKEN') is None:
        print("CHIMP_TOKEN env variable missing")
        exit(1)
    if os.environ.get('WORKFLOW_REPO') is None and os.environ.get('SOURCE_ROOT') is None:
        print("WORKFLOW_REPO env variable missing")
        exit(1)
    print("Polling for available jobs")
    host = os.environ.get("ORCHESTRATION_SERVER", "https://datachimp-api.onrender.com")
    
    r = requests.get(f"{host}/getTasks",
                     headers={'x-token': os.environ.get('CHIMP_TOKEN')})
    r.raise_for_status()
    tasks: typing.List[Task] = r.json()
    global root
    if len(tasks['data']) == 0:
        print("no tasks. Trying again later...")
        root = None
        return
    if root is None:
        root = f'workflow_repo_{int(time.time())}'
    if os.environ.get('SOURCE_ROOT') is None:       
        print("cloning repo")
        subprocess.run(['git', 'clone', os.environ.get("WORKFLOW_REPO"), root])
    task, *_ = tasks['data']
    print(f"received task: {task}")
    _update_task_status(host, task, 'started')
    client = docker.from_env()
    print("initiated docker client")
    image = task.get('image', 'jupyter/tensorflow-notebook')
    print("pulling image")
    client.images.pull(image)
    print("image pulled")
    # Transform code at code_nb_path into a notebook
    script = _task_to_script(task,
                             # TODO: Make it possible to run multiple applets that are selected by the user
                             # just like you can toggle applets within VSCode
                             _get_automations(
                                 task['applets_nb_path'], os.environ.get('SOURCE_ROOT', root)
                             ),
                             os.environ.get('SOURCE_ROOT', root)
                            )
    jupytext.write(jupytext.reads(script, fmt="py:percent"),
                   "data_chimp_notebook.ipynb")
    print("notebook created")
    container = client.containers.run(
        image,
        tty=True,
        command="/bin/bash",
        detach=True
    )
    print("container started")
    # TODO: We'll need to copy more than just the module that has the workflow
    # because the workflow could reference functions defined in other files.
    # We'll probably have to pass the whole codebase. Would be nice if this was
    # just a python package...maybe it could be if we assume usage of nbdev
    with (
        tarfile.open('code.tar', mode='w') as tar_f,
    ):
        tar_f.add(
            f"{os.environ.get('SOURCE_ROOT', root)}/{task['code_nb_path']}",
            f"{task['wf_name']}.py"
        )
        tar_f.add("data_chimp_notebook.ipynb")
    _, output = container.exec_run("mkdir -p data_chimp/source", stream=True)
    for chunk in output:
        print(chunk)
    with open('code.tar') as tar_f:
        container.put_archive('/home/jovyan/data_chimp/source', tar_f)
    container.exec_run(
        "cp data_chimp/source/data_chimp_notebook.ipynb data_chimp/source/data_chimp_notebook_writable.ipynb"
    )
    _, output = container.exec_run(
        "wget https://github.com/getdatachimp/verus/raw/main/data_chimp_executor-0.1.0-py2.py3-none-any.whl",
        stream=True
    )
    for chunk in output:
        print(chunk) 
    _, output = container.exec_run('pip install data_chimp_executor-0.1.0-py2.py3-none-any.whl', stream=True)
    for chunk in output:
        print(chunk) 
    _update_task_status(host, task, 'env_ready')
    print("source copied to container")
    # TODO: Run this function as a subprocess every few seconds so the orchestrator knows the build is still
    # progressing
    _update_task_status(host, task, 'executing')
    container.exec_run(
        'jupyter nbconvert --inplace --allow-errors --execute data_chimp/source/data_chimp_notebook_writable.ipynb'
    )
    print("finished task, sending results to data chimp")
    # Collect notebook results and post to orchestration server
    _bytes, _ = container.get_archive(
        '/home/jovyan/data_chimp/source/data_chimp_notebook_writable.ipynb'
    )
    with open('./sh_bin.tar', 'wb') as f:
        for chunk in _bytes:
            f.write(chunk)
    with tarfile.open('./sh_bin.tar', mode='r') as f:
        f.extractall(path="output")
    with open('output/data_chimp_notebook_writable.ipynb') as f:
        nb = json.load(f)
        r = requests.post(
            f"{host}/updateTask/{task['job_run_id']}",
            json={
                'task_name': task['name'],
                'status': 'done',
                'nb': nb
            },
            headers={
                'x-token': os.environ.get('CHIMP_TOKEN')
            }
        )
        try: 
            r.raise_for_status()
        except Exception as e:
            print("failed to update task status because of error:")
            print(e)
        finally:
            container.stop()
            if not os.environ.get('DC_DEBUG'):
                container.remove()
            print("task container stopped and removed")


def _run_every(func, sec=5):
    s = sched.scheduler(time.time, time.sleep)

    def do_something(sc):
        func()
        sc.enter(sec, 1, do_something, (sc,))

    s.enter(sec, 1, do_something, (s,))
    s.run()

In [44]:
#| export
def activate():
    _run_every(_execute, 5)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()